## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the 'EIN' and 'NAME' columns
application_df = application_df.drop(columns=['EIN', 'NAME'])

# Display the first few rows to confirm the columns are removed
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()

# Display the unique value counts
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
# Check the value counts for APPLICATION_TYPE
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Display the value counts
print(app_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [5]:
# Choose a cutoff value
cutoff_value = 500

# Identify application types to replace
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff_value].index.tolist()

# Replace application types in the DataFrame
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure replacement was successful
print(application_df['APPLICATION_TYPE'].value_counts())

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [6]:
# Check the value counts for CLASSIFICATION
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Display the value counts
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [7]:
# Check the value counts for CLASSIFICATION and filter for counts > 1
classification_counts = application_df['CLASSIFICATION'].value_counts()
filtered_classification_counts = classification_counts[classification_counts > 1]

# Display the filtered value counts
print(filtered_classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [8]:
# Choose a cutoff value
cutoff_value = 1000

# Identify classifications to replace
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
print(application_df['CLASSIFICATION'].value_counts())


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [10]:
# Split the preprocessed data into features (X) and target (y)
X = application_df.drop(columns=['IS_SUCCESSFUL'])  # Drop the target column to form feature set
y = application_df['IS_SUCCESSFUL']  # Target array

# Split the data into training and testing datasets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural network
nn = tf.keras.models.Sequential()

# First hidden layer: specify input dimension and the number of nodes in the layer
# Example: 64 hidden nodes with ReLU activation function
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train.shape[1]))

# Second hidden layer: specify the number of nodes in the layer
# Example: 32 hidden nodes with ReLU activation function
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer: single output node with sigmoid activation for binary classification
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 80)                  │           3,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,981 (23.36 KB)

 Trainable params: 5,981 (23.36 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Compile the model
nn.compile(optimizer='adam',
           loss='binary_crossentropy',
           metrics=['accuracy'])

In [16]:
# Train the model
# Convert X_train and y_train to numpy arrays with the appropriate data type
# X_train = X_train.values.astype('float32')
# y_train = y_train.values.astype('float32')

# # Convert X_test and y_test as well (if needed)
# X_test = X_test.values.astype('float32')
# y_test = y_test.values.astype('float32')

# Now, train the model again
history = nn.fit(X_train, y_train, epochs=100, batch_size=1, validation_data=(X_test, y_test))

Epoch 1/100
27439/27439 ━━━━━━━━━━━━━━━━━━━━ 52s 2ms/step - accuracy: 0.5328 - loss: 0.6911 - val_accuracy: 0.5341 - val_loss: 0.6909
Epoch 2/100
27439/27439 ━━━━━━━━━━━━━━━━━━━━ 59s 2ms/step - accuracy: 0.5350 - loss: 0.6913 - val_accuracy: 0.5341 - val_loss: 0.6912
Epoch 3/100
27439/27439 ━━━━━━━━━━━━━━━━━━━━ 54s 2ms/step - accuracy: 0.5340 - loss: 0.6916 - val_accuracy: 0.5341 - val_loss: 0.6916
Epoch 4/100
27439/27439 ━━━━━━━━━━━━━━━━━━━━ 81s 2ms/step - accuracy: 0.5322 - loss: 0.6918 - val_accuracy: 0.5341 - val_loss: 0.6908
Epoch 5/100
27439/27439 ━━━━━━━━━━━━━━━━━━━━ 82s 2ms/step - accuracy: 0.5310 - loss: 0.6917 - val_accuracy: 0.5341 - val_loss: 0.6908
Epoch 6/100
27439/27439 ━━━━━━━━━━━━━━━━━━━━ 81s 2ms/step - accuracy: 0.5283 - loss: 0.6918 - val_accuracy: 0.5341 - val_loss: 0.6908
Epoch 7/100
27439/27439 ━━━━━━━━━━━━━━━━━━━━ 86s 2ms/step - accuracy: 0.5334 - loss: 0.6911 - val_accuracy: 0.5341 - val_loss: 0.6909
Epoch 8/100
27439/27439 ━━━━━━━━━━━━━━━━━━━━ 82s 2ms/step - ac

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - 3ms/step - accuracy: 0.5612 - loss: 0.7946
Loss: 0.7946118116378784, Accuracy: 0.5612244606018066


In [18]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')

# Confirm the model has been saved
print("Model saved to AlphabetSoupCharity.h5")

Model saved to AlphabetSoupCharity.h5
